In [1]:
import os


In [2]:
%pwd

'/Users/ronaksatpute/Projects/MLOps/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/ronaksatpute/Projects/MLOps'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from src.MLProject.constants import *
from src.MLProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH,
        ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from src.MLProject import logger
from src.MLProject.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(
                f"File: {filename} downloaded with following info: \n{headers}"
            )
        else:
            logger.info(
                f"File already exists of size: {get_size(Path(self.config.local_data_file))}"
            )

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info(f"File extracted at location: {unzip_path}")   

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()

    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-12-04 16:26:22,385:INFO:common:YAML file: configs/config.yaml loaded successfully]
[2025-12-04 16:26:22,386:INFO:common:YAML file: params.yaml loaded successfully]
[2025-12-04 16:26:22,386:INFO:common:YAML file: schema.yaml loaded successfully]
[2025-12-04 16:26:22,386:INFO:common:Directory created at: artifacts]
[2025-12-04 16:26:22,387:INFO:common:Directory created at: artifacts/data_ingestion]
[2025-12-04 16:26:22,387:INFO:common:File size for artifacts/data_ingestion/data.zip is 26 KB]
[2025-12-04 16:26:22,387:INFO:172361131:File already exists of size: 26 KB]
[2025-12-04 16:26:22,388:INFO:172361131:File extracted at location: artifacts/data_ingestion]
